<a href="https://colab.research.google.com/github/josee-pp/03MIAR-Algoritmos-de-Optimizacion-2026/blob/main/TRABAJO_PRACTICO/Trabajo_Practico_Jos%C3%A9_Enrique_Pinz%C3%B3n_Parra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TRABAJO PRÁCTICO**

### Alumno: José Enrique Pinzón Parra
### Asignatura: MIAR03 - Algoritmos de Optimización

Enlaces:
* GitHub: https://github.com/josee-pp/03MIAR-Algoritmos-de-Optimizacion-2026/blob/master/TRABAJO_PRACTICO/Trabajo_Practico_Jos%C3%A9_Enrique_Pinz%C3%B3n_Parra.ipynb
* Google Colab: https://colab.research.google.com/drive/1YKaqSMnxA57Ug4TNCdCNqa3s3OTdyuOL?usp=sharing

Para esta entrega se ha escogido resolver el problema #3 "Generación de Tribunales". Citando el enunciado del problema, se pide lo siguiente:

* *Se precisa configurar tribunales de evaluación para un grupo de 15 alumnos que desean presentar su Trabajo Fin de Máster (TFM).*
* *Cada tribunal está compuesto por tres profesores, cada uno desempeñando uno de los siguientes roles: Presidente, Secretario o Vocal.*
* *Los profesores han indicado su disponibilidad horaria para participar en los tribunales de 15h a 21h durante la semana del 15 al 19 de abril:*
    * *Número de profesores : 10*
    * *Número de tribunales : 15*

*Disponibilidad/Roles : https://bit.ly/41QWk8o*
- *1 indica que profesor tiene disponibilidad*
- *0 en caso contrario*

*Hay 15 alumnos, por lo que se deben configurar 15 tribunales buscando la configuración más equilibrada posible en cuanto a la cantidad de tribunales asignados a cada profesor, es decir, evitando que un profesor tenga muchos tribunales y otros pocos.*

*Obviamente ningún profesor puede asistir a dos tribunales a la misma fecha/hora y no puede ser convocado a un tribunal al que no tiene disponibilidad.*



### **0. Importar librerías**

In [11]:
import pandas as pd
from collections import defaultdict
import math
import time
import random

### **1. Preparación de los datos**

En este bloque se cargan los datos de disponibilidad horaria y roles de los profesores directamente desde Google Sheets, garantizando la reproducibilidad del notebook. A continuación, se reorganizan las columnas de disponibilidad generando explícitamente todas las combinaciones día–hora consideradas en el problema. Posteriormente, los roles de los profesores se transforman en variables binarias mediante codificación one-hot, permitiendo identificar de forma clara qué roles puede desempeñar cada profesor. Finalmente, ambos conjuntos de datos se integran en un único dataset binario que sirve como base para el algoritmo de optimización aplicado posteriormente.

In [12]:
sheet_id = "1nGeFXiDXH7Cy-ed8oUdrDfAQxQOzJ_MmJ5fqOSY9o48"
# Carga de la hoja de disponibilidad
disponibilidad = pd.read_csv(
    f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid=0",
    index_col=0,
    header=1)
# Carga de la hoja de roles
roles = pd.read_csv(
    f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid=1701339457")

# Construcción de combinaciones día-hora
dias = ['15', '16', '17', '18', '19']
horas = list(map(str, disponibilidad.columns.tolist()[1:8]))
dia_hora = []
for dia in dias:
    for hora in horas:
        dia_hora.append(dia + '-' + hora)
dia_hora.insert(0, 'Profesor')
disponibilidad.columns = dia_hora

# Disponibilidad
print("Hoja de disponibilidad")
display(disponibilidad)
print("")

# Conversión de los roles a variables binarias (one-hot encoding)
roles_dummies = roles['ROL'].str.get_dummies(sep=',')
roles_df_processed = pd.concat([roles['Profesor'], roles_dummies], axis=1)

# Roles
print("Hoja de roles")
display(roles_df_processed)
print("")

# Dataset final
df = disponibilidad.merge(roles_df_processed, on='Profesor')
print("Dataset final")
display(df)

Hoja de disponibilidad


,Profesor,15-15,15-16,15-17,15-18,15-19,15-20,15-21,16-15,16-16,...,18-19,18-20,18-21,19-15,19-16,19-17,19-18,19-19,19-20,19-21
1,RRD,0,1,1,1,0,1,1,1,0,...,1,1,1,1,1,1,1,1,0,0
2,QYV,1,1,1,1,0,0,0,0,1,...,1,1,1,1,1,1,1,1,1,1
3,LHL,0,0,1,1,0,1,1,1,1,...,1,0,1,0,1,1,0,1,0,1
4,HLC,1,0,1,0,1,1,0,1,0,...,0,1,1,1,1,1,1,1,1,0
5,MSB,1,1,0,1,0,1,1,1,1,...,0,1,1,1,0,1,1,1,1,0
6,PMQ,1,1,1,1,1,0,0,1,1,...,0,1,1,1,1,1,0,1,0,1
7,QWF,0,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,0,1
8,EBB,1,1,1,1,1,0,0,1,1,...,1,1,1,0,1,1,1,0,1,0
9,IOE,1,0,1,1,0,1,0,0,1,...,1,1,0,1,0,1,1,1,1,1
10,IOA,1,1,0,1,1,0,1,1,0,...,1,1,1,1,1,1,0,0,0,1



Hoja de roles


,Profesor,P,S,V
0,RRD,1,1,1
1,QYV,1,1,1
2,LHL,1,0,1
3,HLC,0,1,1
4,MSB,1,1,1
5,PMQ,1,1,1
6,QWF,0,1,1
7,EBB,0,1,1
8,IOE,1,1,1
9,IOA,1,1,1



Dataset final


,Profesor,15-15,15-16,15-17,15-18,15-19,15-20,15-21,16-15,16-16,...,19-15,19-16,19-17,19-18,19-19,19-20,19-21,P,S,V
0,RRD,0,1,1,1,0,1,1,1,0,...,1,1,1,1,1,0,0,1,1,1
1,QYV,1,1,1,1,0,0,0,0,1,...,1,1,1,1,1,1,1,1,1,1
2,LHL,0,0,1,1,0,1,1,1,1,...,0,1,1,0,1,0,1,1,0,1
3,HLC,1,0,1,0,1,1,0,1,0,...,1,1,1,1,1,1,0,0,1,1
4,MSB,1,1,0,1,0,1,1,1,1,...,1,0,1,1,1,1,0,1,1,1
5,PMQ,1,1,1,1,1,0,0,1,1,...,1,1,1,0,1,0,1,1,1,1
6,QWF,0,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,1,0,1,1
7,EBB,1,1,1,1,1,0,0,1,1,...,0,1,1,1,0,1,0,0,1,1
8,IOE,1,0,1,1,0,1,0,0,1,...,1,0,1,1,1,1,1,1,1,1
9,IOA,1,1,0,1,1,0,1,1,0,...,1,1,1,0,0,0,1,1,1,1


### **2. Definición de conjuntos, parámetros y funciones**

**a. Espacio de soluciones:**

Está conformado por todas las posibles configuraciones válidas de los tribunales para las 15 presentaciones de TFM. Cada solución consiste en asignar para cada tribunal una franja temporal y tres profesores distintos que desempeñan los roles de Presidente (P), Secretario (S) y Vocal (V).

Dichas franjas temporales se tratan del conjunto de combinaciones día-hora, donde los días van del 15 al 19 de abril, y las horas de 15:00 a 21:00, donde cada presentación dura 1 hora.

Una solución puede representarse como la tupla:

$$
(t, f, p_P, p_S, p_V)
$$

donde:

- $t \in \{1, \dots, 15\}$ identifica el tribunal,
- $f$ es una franja temporal válida (día–hora),
- $p_P, p_S, p_V$ son profesores distintos asignados a los roles de P, S y V.

Es evidente que las asignaciones solo serán válidas si cumplen las restricciones puestas en el problema.

**b. Función objetivo**

El objetivo es equilibrar la carga de trabajo entre los profesores. Sea $c_p$ el número total de tribunales asignados al profesor $p$, se define el desequilibrio de carga como la diferencia entre la máxima y la mínima carga docente:

$$
\max_{p}(c_p) - \min_{p}(c_p)
$$

La función objetivo consiste en minimizar dicho desequilibrio, buscando una distribución lo más equitativa posible de los tribunales entre los profesores:

$$
\min \left( \max_{p}(c_p) - \min_{p}(c_p) \right)
$$

**c. Implementación de las restricciones**

Garantizan que todas las soluciones generadas sean factibles y cumplan las condiciones del problema.

* **R1:** Cada tribunal debe celebrarse en una única franja temporal (día–hora):

* **R2:** Cada tribunal debe contar con exactamente 1 P, 1 S y 1 V.

* **R3:** Un profesor solo puede ser asignado a un tribunal si dicho tribunal se celebra en la franja correspondiente.

* **R4:** Un profesor no puede desempeñar más de un rol en el mismo tribunal.

* **R5:** Un profesor no puede participar en más de un tribunal en la misma franja horaria.

* **R6:** Quedan prohibidas asignaciones en las que el profesor no tenga permitido desempeñar algún rol concreto y/o no esté disponible en alguna franja horaria determinada.

In [13]:
# Lista de profesores disponibles en el problema
profesores = df['Profesor'].tolist()

# Identificación de las franjas temporales (columnas día-hora)
franjas = [c for c in df.columns if '-' in c]

# Roles posibles en cada tribunal
roles = ['P', 'S', 'V']

# Número total de tribunales a asignar
N_TRIBUNALES = 15

# -------------------------------
# Funciones auxiliares de consulta
# -------------------------------

# Comprueba si el profesor p está disponible en la franja f
def disponible(p, f):
    return df.loc[df['Profesor'] == p, f].iloc[0] == 1

# Comprueba si el profesor p puede desempeñar el rol r
def rol_ok(p, r):
    return df.loc[df['Profesor'] == p, r].iloc[0] == 1

# -------------------------------
# Construcción del DataFrame final
# -------------------------------

# Convierte una solución interna en un DataFrame estructurado
def construir_dataframe(sol):
    df_out = pd.DataFrame({
        'Tribunal': list(range(1, N_TRIBUNALES + 1)),
        'Fecha': [None] * N_TRIBUNALES,
        'Hora': [None] * N_TRIBUNALES,
        'Presidente': [None] * N_TRIBUNALES,
        'Secretario': [None] * N_TRIBUNALES,
        'Vocal': [None] * N_TRIBUNALES
    })

    # Relleno del DataFrame a partir de la solución
    for t, (franja, roles_dict) in sol.items():
        fecha, hora = map(int, franja.split('-'))
        idx = df_out.index[df_out['Tribunal'] == t][0]

        df_out.loc[idx, ['Fecha', 'Hora']] = fecha, hora
        df_out.loc[idx, 'Presidente'] = roles_dict['P']
        df_out.loc[idx, 'Secretario'] = roles_dict['S']
        df_out.loc[idx, 'Vocal'] = roles_dict['V']

    return df_out

# -------------------------------
# Función objetivo
# -------------------------------

# Calcula el desequilibrio de carga entre profesores
def desequilibrio(carga):
    if not carga:
        return 0
    return max(carga.values()) - min(carga.values())

### **3. Método utilizado: Ramificación y poda**

Enfrentamos el problema inicialmente aplicando Ramificación y Poda al tratarse de una técnica exacta adecuada para problemas con variables binarias (se asigna o no se asigna, 1 y 0) y un espacio de soluciones finito (15) discreto, garantizando una solución óptima mientras explora sistemáticamente el espacio de soluciones y descarta ramas que no pueden mejorar la solución encontrada hasta el momento. Debido a eso, tiene un elevado coste computacional que limita su escalabilidad.

En la función diseñada se van construyendo las soluciones de forma incremental, asignando los tribunales uno a uno mediante un proceso recursivo.

En cada nivel de la recursión se generan todas las combinaciones factibles de franja horaria y profesores para los roles P, S y V, verificando en ese mismo momento las restricciones de disponibilidad, compatibilidad de roles y ausencia de solapamientos temporales.

Cada asignación válida da lugar a una nueva rama del árbol de búsqueda. Para reducir el espacio explorado, se utiliza una cota inferior basada en el desequilibrio de carga actual; si esta cota no puede mejorar la mejor solución encontrada, la rama se poda y no se continúa explorando.

In [14]:
best_solution = None
best_value = math.inf

def branch_and_bound(t,estado,carga,ocupado,profesores,franjas,roles,N_TRIBUNALES):

    # Mejor solución encontrada hasta el momento
    global best_solution, best_value

    # -------------------------------
    # Caso base
    # Todos los tribunales asignados
    # -------------------------------
    if t > N_TRIBUNALES:
        val = desequilibrio(carga)
        if val < best_value:
            best_value = val
            best_solution = estado.copy()
        return

    # -------------------------------
    # Poda por cota inferior
    # Si el desequilibrio actual ya no puede mejorar la mejor solución
    # -------------------------------
    if carga and desequilibrio(carga) >= best_value:
        return

    # -------------------------------
    # Ramificación
    # Se prueban todas las combinaciones válidas de franja y roles P, S y V
    # -------------------------------
    for franja in franjas:
        for P in profesores:
            # Restricción: rol permitido, disponibilidad y no solapamiento
            if not (rol_ok(P, 'P') and disponible(P, franja)) or franja in ocupado[P]:
                continue

            for S in profesores:
                # Restricción: profesor distinto, rol permitido, disponibilidad y no solapamiento
                if S == P or not (rol_ok(S, 'S') and disponible(S, franja)) or franja in ocupado[S]:
                    continue

                for V in profesores:
                    # Restricción: profesor distinto, rol permitido, disponibilidad y no solapamiento
                    if V in {P, S} or not (rol_ok(V, 'V') and disponible(V, franja)) or franja in ocupado[V]:
                        continue

                    # -------------------------------
                    # Aplicar la asignación
                    # -------------------------------
                    estado[t] = (franja, {'P': P, 'S': S, 'V': V})
                    for p in (P, S, V):
                        carga[p] += 1
                        ocupado[p].add(franja)

                    # Llamada recursiva al siguiente tribunal
                    branch_and_bound(t+1,estado,carga,ocupado,profesores,franjas,roles,N_TRIBUNALES)

                    # -------------------------------
                    # Backtracking:
                    # se deshacen los cambios realizados
                    # -------------------------------
                    for p in (P, S, V):
                        carga[p] -= 1
                        ocupado[p].remove(franja)
                    del estado[t]

In [15]:
# Parámetros de inicio
estado_inicial = {}
carga_inicial = defaultdict(int)
ocupado_inicial = defaultdict(set)

start = time.time()

# Llamamos a la función y obtenemos la mejor solución
branch_and_bound(1,estado_inicial,carga_inicial,ocupado_inicial,profesores,franjas,roles,N_TRIBUNALES)

time = time.time() - start

solution = construir_dataframe(best_solution)

print(f"Tiempo de ejecución: {time:.2f} segundos")
display(solution)

Tiempo de ejecución: 188.36 segundos


,Tribunal,Fecha,Hora,Presidente,Secretario,Vocal
0,1,15,15,QYV,HLC,MSB
1,2,15,15,PMQ,EBB,IOE
2,3,15,16,RRD,QYV,MSB
3,4,15,16,PMQ,QWF,EBB
4,5,15,18,RRD,IOA,LHL
5,6,15,17,LHL,HLC,QWF
6,7,16,21,RRD,IOE,IOA
7,8,15,17,QYV,PMQ,EBB
8,9,15,18,MSB,QWF,IOE
9,10,16,15,LHL,HLC,IOA


El resultado final muestra una asignación factible y equilibrada, cumpliendo todas las restricciones del problema: disponibilidad horaria, compatibilidad de roles, ausencia de solapamientos y unicidad de roles por tribunal. Esto confirma que, a pesar de la complejidad inherente, el enfoque es adecuado y consistente para instancias de este tamaño, especialmente cuando se prioriza una solución óptima.

En relación a la **complejidad del algoritmo**, en el peor caso no se logra podar ninguna rama y se exploran todas las combinaciones posibles del espacio de soluciones. Para cada tribunal se consideran todas las franjas temporales y todas las posibles ternas ordenadas de profesores distintos para los roles $(P, S, V)$.

La complejidad en el peor caso es:

$$
\mathcal{O}\left( (|F| \cdot |P|^3)^{T} \right)
$$

donde:
- $ |F| $ es el número de franjas temporales posibles,
- $ |P| $ es el número de profesores,
- $( T )$ es el número de tribunales.

Este comportamiento es de tipo exponencial, reflejando la naturaleza combinatoria del problema.

En el mejor escenario, por otro lado, la poda elimina la mayoría de las ramas del árbol de búsqueda en niveles muy tempranos gracias a una cota inferior efectiva. En este caso, el número de nodos explorados se reduce drásticamente.

La complejidad en el mejor caso puede aproximarse como:

$$
\mathcal{O}(T \cdot |F| \cdot |P|^3)
$$

La complejidad teórica sigue siendo exponencial, aunque la poda puede hacer que el tiempo de ejecución crezca de forma mucho más moderada en la práctica.